# Log grouping -

In [ ]:
!pip install gower

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
import numpy as np

#!pip install gensim

In [ ]:
def convert_to_vector (sentence, modelv):
    vset = []
    numwords = 0

    for word in sentence:
        print("word :", word)
        try:
            if (numwords == 0):
                vset  = modelv.wv.get_vector(word, norm=True)
            else:
                vset = np.add(vset,modelv.wv.get_vector(word, norm=True))
            numwords += 1
        except:
           pass
    print(vset)
    return(np.asarray(vset)/numwords)

## Dataset: Wine reviews

In [ ]:
import pandas as pd

# 100,000 wine reviews

wine_df = pd.read_csv("wine.csv")
reviews = wine_df["description"].values[:10000]

#### Sample wine review

In [ ]:
len(reviews)

## Preprocess the corpus (text)

In [ ]:
# A list of stopp words and punctuation to be removed
stpwrds = stopwords.words('english') + list(string.punctuation)

In [ ]:
processed_reviews = []

for review in  reviews:
    processed_reviews.append([word 
                                for word in word_tokenize(review.lower())
                                if word 
                                not in stpwrds]) 

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
# Preprocess 100,000 wine reviews
modelvec = Word2Vec(sentences=processed_reviews, vector_size=40, # or use size, depending on version
                 window=40, min_count=10, sg=1)
sentences = []
 


In [ ]:
for review in processed_reviews:
    sentences.append(convert_to_vector(review,modelvec))

In [ ]:
VecX =  np.array(sentences)

In [ ]:
#how to find K 
import matplotlib.pyplot as plt
within_cluster_ss = []
for index in range(1,6):
    kmeans =  cluster.KMeans(n_clusters=index,n_init=10,random_state=42 )
    #random set each time with 42
    kmeans.fit(VecX)
    within_cluster_ss.append(kmeans.inertia_)
plt.plot(range(1,6), within_cluster_ss)
plt.xlabel('Values for K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

# we have found k = 3 using elbow method for kmeans

In [ ]:
 #Agglomerative clustering is sophisticated version of hierarchical clustering. We see here to that 3 is "choice"  based on 
#silhouette score.    

In [ ]:
#Now we need to determine and check K for hierarchy
hl = hierarchy.linkage(VecX,'ward')
#This is a beautiful and yet simple method - ward to determine distance
dng = hierarchy.dendrogram(hl)
plt.xlim(0,10)
plt.ylim(0,0.5)
plt.title ("Dendrogram")
plt.ylabel("distance")

In [ ]:
from numpy import unique
from numpy import where
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot
from sklearn.metrics import silhouette_score 

agc = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='ward')
yhat_agc = agc.fit_predict(VecX)
#Now game over
print(yhat_agc)
yhat = agc.fit(VecX)
# retrieve unique clusters
clusters = unique(yhat)
# Calculate cluster validation metrics
score_AGclustering_s = silhouette_score(VecX, yhat.labels_, metric='euclidean')
print('Silhouette Score: ',  score_AGclustering_s)
 

In [ ]:
for index,sent in enumerate(processed_reviews):
    print( str(yhat_agc[index]), ":::", str(sent))
        



# Using Gower matrix

In [ ]:
import gower
 

In [ ]:
distance_matrix = gower.gower_matrix(VecX)

In [ ]:
distance_matrix

In [ ]:
new_agc = AgglomerativeClustering(n_clusters=3, metric='precomputed', linkage='average')
clusters =new_agc.fit_predict(distance_matrix)

In [ ]:
yhat = new_agc.fit(distance_matrix)
score_AGclustering_s = silhouette_score(distance_matrix, yhat.labels_, metric='precomputed')
print('Silhouette Score: ',  score_AGclustering_s)

